# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
import pandas as pd
import numpy as np

import statsmodels.formula.api as smf
import statsmodels.api as sm
import patsy

from sklearn.model_selection import train_test_split
from sklearn.tree            import DecisionTreeClassifier
from sklearn.metrics         import r2_score

In [2]:
df = pd.read_csv('previsao_de_renda.csv').drop(columns = ['Unnamed: 0','data_ref','id_cliente'])

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   sexo                   15000 non-null  object 
 1   posse_de_veiculo       15000 non-null  bool   
 2   posse_de_imovel        15000 non-null  bool   
 3   qtd_filhos             15000 non-null  int64  
 4   tipo_renda             15000 non-null  object 
 5   educacao               15000 non-null  object 
 6   estado_civil           15000 non-null  object 
 7   tipo_residencia        15000 non-null  object 
 8   idade                  15000 non-null  int64  
 9   tempo_emprego          12427 non-null  float64
 10  qt_pessoas_residencia  15000 non-null  float64
 11  renda                  15000 non-null  float64
dtypes: bool(2), float64(3), int64(2), object(5)
memory usage: 1.2+ MB


1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

### 1.Separe a base em treinamento e teste (25% para teste, 75% para treinamento).

In [5]:
df_train, df_test = train_test_split(df, test_size=0.25, random_state=100)

### 2.Rode uma regularização ridge com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o  𝑅2 na base de testes. Qual o melhor modelo?

In [6]:
modelo = '''np.log(renda) ~ 
                C(sexo, Treatment("F")) 
                + C(posse_de_veiculo, Treatment(False))
                + C(posse_de_imovel, Treatment(True))
                + idade
                + tempo_emprego '''

md = smf.ols(modelo, data = df_test)

alphas = [0,0.001,0.005,0.01,0.05,0.1]

for i in alphas:
    reg = md.fit_regularized(method = 'elastic_net' 
                             , refit = True
                             , L1_wt = 0.0001
                             , alpha = i)
    print(reg.rsquared)

0.3439940760353828
0.3439940760353828
0.3439940760353828
0.3439940760353828
0.3439940760353828
0.3439940760353828


### 3.Faça o mesmo que no passo 2, com uma regressão LASSO. Qual método chega a um melhor resultado?

In [7]:
modelo = '''np.log(renda) ~ 
                C(sexo, Treatment("F")) 
                + C(posse_de_veiculo, Treatment(False))
                + C(posse_de_imovel, Treatment(True))
                + idade
                + tempo_emprego '''
md = smf.ols(modelo, data = df_test)

alphas = [0,0.001,0.005,0.01,0.05,0.1]

for i in alphas:
    reg = md.fit_regularized(method = 'elastic_net' 
                             , refit = True
                             , L1_wt = 1
                             , alpha = i)
    print(reg.rsquared)

0.3439940760353828
0.3439940760353828
0.3426013570257954
0.3426013570257954
0.3402929648471522
0.3402929648471522


**Resposta:** Os alphas de valor menor (0 e 0.001) chegam a R² maiores (que são iguais aos R² via regularização de ridge).

### 4.Rode um modelo stepwise. Avalie o  𝑅2 na vase de testes. Qual o melhor resultado?

O código stepwise fornecido em aula não suporta valores booleanos ou de object e não pode ter NA. Então preciso:

1- Converter os booleanos em int e os objects em dummies.

2- Eliminar os Na.

In [8]:
df_test = pd.get_dummies(df_test, columns=['sexo','tipo_renda','educacao','estado_civil','tipo_residencia'], drop_first=True)

In [9]:
df_test = df_test.astype(float).dropna()

In [10]:
X = df_test.drop('renda',axis=1)
y = pd.DataFrame(np.log(df_test.renda))

In [11]:
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.05, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded, dtype=np.dtype('float64'))
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.index[new_pval.argmin()]
            included.append(best_feature)
            changed=True
            if verbose:
                 print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        print("#############")
        print(included)
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [12]:
variaveis = stepwise_selection(X, y)

Add  tempo_emprego                  with p-value 3.50864e-125
#############
['tempo_emprego']
Add  sexo_M                         with p-value 1.00366e-152
#############
['tempo_emprego', 'sexo_M']
Add  tipo_renda_Empresário          with p-value 8.32885e-10
#############
['tempo_emprego', 'sexo_M', 'tipo_renda_Empresário']
Add  idade                          with p-value 2.28563e-08
#############
['tempo_emprego', 'sexo_M', 'tipo_renda_Empresário', 'idade']
Add  educacao_Superior completo     with p-value 1.16568e-07
#############
['tempo_emprego', 'sexo_M', 'tipo_renda_Empresário', 'idade', 'educacao_Superior completo']
Add  posse_de_imovel                with p-value 0.0015895
#############
['tempo_emprego', 'sexo_M', 'tipo_renda_Empresário', 'idade', 'educacao_Superior completo', 'posse_de_imovel']
Add  tipo_residencia_Estúdio        with p-value 0.00947258
#############
['tempo_emprego', 'sexo_M', 'tipo_renda_Empresário', 'idade', 'educacao_Superior completo', 'posse_de_imovel', '

### 5.Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?

In [13]:
df_test.rename(columns={'educacao_Superior completo': 'educacao_Superior_completo'}, inplace=True)

In [14]:
modelo = '''np.log(renda) ~ 
                tempo_emprego 
                + sexo_M
                + tipo_renda_Empresário
                + idade
                + educacao_Superior_completo
                + posse_de_imovel
                + tipo_residencia_Estúdio
                + posse_de_veiculo
                '''
smf.ols(modelo, data = df_test).fit().rsquared

0.3590164987902693

**Resposta:** O melhor resultado de todos foi o Stepwise.

### 6.Partindo dos modelos que você ajustou, tente melhorar o  𝑅2 na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.

In [15]:
modelo = '''np.log(renda) ~ 
                tempo_emprego
                + sexo_M
                + tipo_renda_Empresário
                + np.log(idade)
                + educacao_Superior_completo
                + posse_de_imovel
                + tipo_residencia_Estúdio
                + posse_de_veiculo
                '''
smf.ols(modelo, data = df_test).fit().rsquared

0.3596625771124603

**Resposta:** Transformar a idade em log teve um efeito positivo no R². Experimentei fazer o mesmo com tempo_emprego, mas teve um efeito negativo.

### 7.Ajuste uma árvore de regressão e veja se consegue um  𝑅2 melhor com ela.

In [16]:
# Importando e tratando base
df = pd.read_csv('previsao_de_renda.csv').drop(columns = ['Unnamed: 0','data_ref','id_cliente'])
df = pd.get_dummies(df, columns=['sexo','tipo_renda','educacao','estado_civil','tipo_residencia'], drop_first=True)
df = df.dropna()

#Dividindo base
X = df.drop('renda',axis=1)
y = pd.DataFrame(np.log(df.renda).astype(int)) 
#essas transformações são importantes para comparar com as outras regressões que fizesmos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=100)

# Treinando a árvore
clf = DecisionTreeClassifier(random_state=100, min_samples_leaf=5, max_depth=100)
reg = clf.fit(X_train,y_train)

# Testando e avaliando
y_pred = reg.predict(X_test)

# Calcular o R²
r2_score(y_test, y_pred)

0.05391221129881618

**Resposta:** Obtivemos o maior R² até agora, mas com risco de overfitting.